In [42]:
# install chromium, its driver, and selenium
!apt -qq update
!apt -qq install chromium-chromedriver
!pip -qq install selenium
# !pip install -qq python-pptx

41 packages can be upgraded. Run 'apt list --upgradable' to see them.
chromium-chromedriver is already the newest version (87.0.4280.66-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [43]:
%cd /content/drive/My Drive/Colab Notebooks/AMEX/ResponseEngineDataUpdate_v2

from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np
import os
import glob
import calendar

import sys
sys.path.append('01_utils')
# from getdata import *
# from processdata import *
from hlpr import *
from config import *

import warnings
warnings.filterwarnings('ignore')

/content/drive/My Drive/Colab Notebooks/AMEX/ResponseEngineDataUpdate_v2


In [44]:
mastermap_ = mastermap()

In [45]:
def process_google_trend():
  '''
  Process Google historical and current data for the trend tab.
  '''
  # temp market map
  mkt_map = {'AU':'Australia','GB':'United Kingdom','US':'United States','CA':'Canada','MX':'Mexico'}
  
  # import google search results
  GOOGLE_PATH = os.path.join(DATA_PATH, 'google')
  
  df_list = []
  for f in os.listdir(GOOGLE_PATH):
    for d in os.listdir(os.path.join(GOOGLE_PATH, f)):
      df = pd.read_csv(os.path.join(GOOGLE_PATH, f, d))
      df_list.append(df)

  # combine
  df = pd.concat(df_list, sort=False).reset_index(drop=True)
  
  # adjust columns
  df['date'] = pd.to_datetime(df['date'])
  df = df[df['date']<df['date'].max()].reset_index(drop=True)
  df.rename(columns={'variable':'term'}, inplace=True)
  
  # join to category map
  df_join = mastermap_['google_categories'].merge(df, how='left')

  r = google_weekday(df_join, mkt_map)

  return r

In [48]:
def google_weekday(df, mkt_map):
    '''
    Export data with weekday added.
    '''
    # filter to >= 2020 and US only
    df = df[df['date'].dt.year >= 2020]
    df['Year No'] = df['date'].dt.year

    # add weekday
    df['weekday'] = df['date'].apply(lambda x: x.strftime('%A'))

    # add market name
    df['market_name'] = df['market'].apply(lambda x: mkt_map[x] if x in mkt_map.keys() else None)
    df = df[-df['market_name'].isna()]
    df.reset_index(drop=True)

    # rename
    df.rename(columns={'term':'variable'}, inplace=True)

    # export
    df.to_csv(os.path.join(OUTPUT_PATH, 'google_weekday.csv'), index=False)

    return df

In [49]:
r = process_google_trend()

In [ ]:
r.head()

,Master Category,Category,term,date,market,value,market_name,weekday
1275,Additional,Cultural Trends,baking,2020-01-01,US,34.0,United States,Wednesday
1276,Additional,Cultural Trends,baking,2020-01-02,US,24.0,United States,Thursday
1277,Additional,Cultural Trends,baking,2020-01-03,US,34.0,United States,Friday
1278,Additional,Cultural Trends,baking,2020-01-04,US,33.0,United States,Saturday
1279,Additional,Cultural Trends,baking,2020-01-05,US,35.0,United States,Sunday
